In [1]:
#-- required imports
import tensorflow as tf
from tensorflow.keras import regularizers as rg
import gc
import pickle as pkl

import matplotlib.pyplot as plt
import os
import numpy as np
import time

import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
features_path = "/kaggle/input/timitpreprocessed/features.pkl"
labels_path = "/kaggle/input/timitpreprocessed/labels.pkl"

with open(features_path, 'rb') as pickle_file:
    features = pkl.load(pickle_file)
    
with open(labels_path, 'rb') as pickle_file:
    labels = pkl.load(pickle_file)

In [3]:
map_phonemes_61_to_39 = {
            'iy':'iy',  'ih':'ih',   'eh':'eh',  'ae':'ae',    'ix':'ih',  'ax':'ah',   'ah':'ah',  'uw':'uw',
            'ux':'uw',  'uh':'uh',   'ao':'aa',  'aa':'aa',    'ey':'ey',  'ay':'ay',   'oy':'oy',  'aw':'aw',
            'ow':'ow',  'l':'l',     'el':'l',  'r':'r',      'y':'y',    'w':'w',     'er':'er',  'axr':'er',
            'm':'m',    'em':'m',     'n':'n',    'nx':'n',     'en':'n',  'ng':'ng',   'eng':'ng', 'ch':'ch',
            'jh':'jh',  'dh':'dh',   'b':'b',    'd':'d',      'dx':'dx',  'g':'g',     'p':'p',    't':'t',
            'k':'k',    'z':'z',     'zh':'sh',  'v':'v',      'f':'f',    'th':'th',   's':'s',    'sh':'sh',
            'hh':'hh',  'hv':'hh',   'pcl':'h#', 'tcl':'h#', 'kcl':'h#', 'qcl':'h#','bcl':'h#','dcl':'h#',
            'gcl':'h#','h#':'h#',  '#h':'h#',  'pau':'h#', 'epi': 'h#','nx':'n',   'ax-h':'ah','q':'h#' 
        }

phonemes_list_61 = list(map_phonemes_61_to_39.keys())
phonemes_list_39 = list(set(map_phonemes_61_to_39.values()))

label_to_phoneme39 = {}
phoneme39_to_label = {}
for index,phoneme in enumerate(phonemes_list_39):
    label_to_phoneme39[phoneme] = index + 1
    phoneme39_to_label[index + 1] = phoneme

map_phonemes_39_to_61 = {}

for phoneme61,phoneme39 in map_phonemes_61_to_39.items():
    if not phoneme39 in map_phonemes_39_to_61:
        map_phonemes_39_to_61[phoneme39] = []
    map_phonemes_39_to_61[phoneme39].append(phoneme61)

In [4]:
print(features.shape)
print(labels.shape)

(1421707, 192)
(1421707, 39)


In [5]:
####--------------Model Training----------------------###   
n_mels = 64
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1024, input_shape=[n_mels*3],activation=tf.nn.relu),
    tf.keras.layers.Dense(units=1024,activation=tf.nn.relu),
    tf.keras.layers.Dense(units=1024,activation=tf.nn.relu),
    tf.keras.layers.Dense(units=39,activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
gc.collect()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              197632    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_3 (Dense)             (None, 39)                39975     
                                                                 
Total params: 2,336,807
Trainable params: 2,336,807
Non-trainable params: 0
_________________________________________________________________


103

In [6]:
performance_dnn = {}

import numpy as np
from tensorflow.keras.callbacks import Callback
import Levenshtein as lev

class PERCallback(Callback):
    def __init__(self, X_val, y_val, phoneme_mapping):
        self.X_val = X_val
        #self.X_val = self.X_val.reshape(self.X_val.shape[0], 1, self.X_val.shape[1])
        self.y_val = y_val
        self.phoneme_mapping = phoneme_mapping

    def on_epoch_end(self, epoch, logs=None):
        # Get the model predictions
        predictions = self.model.predict(self.X_val)
        
        # Convert one-hot encoded vectors to phoneme sequences
        reference_phonemes = [self.one_hot_to_phoneme(vec) for vec in self.y_val]
        predicted_phonemes = [self.one_hot_to_phoneme(vec) for vec in predictions]

        # Calculate PER
        per = self.calculate_per(" ".join(reference_phonemes), " ".join(predicted_phonemes))
        performance_dnn[epoch] = per
        # Print PER
        print(f'\n Phoneme Error Rate after epoch {epoch}: {per}%')

    def one_hot_to_phoneme(self, one_hot_vector):
        index = np.argmax(one_hot_vector)
        return self.phoneme_mapping.get(index-1, "")
    
    import Levenshtein as lev
    
    def calculate_per(self, reference, hypothesis):
        distance = lev.distance(reference, hypothesis)
        per = distance / len(reference)
        return per * 100

In [7]:
start_time = time.time()

history = model.fit(
    features[:1137000],labels[:1137000],epochs=30,
    batch_size=512, verbose=1,
    validation_data=(features[1137000:],labels[1137000:]),
    validation_batch_size=128,
    callbacks=[PERCallback(features[1137000:],labels[1137000:], phoneme39_to_label)]
)

end_time = time.time()

# Calculate the training time
training_time = end_time - start_time
print(f"Training time: {training_time:.2f} seconds")

Epoch 1/30
8898/8898 [==============================] - 16s 2ms/step

 Phoneme Error Rate after epoch 0: 30.29914159823858%
2221/2221 [==============================] - 96s 40ms/step - loss: 1.9920 - accuracy: 0.4626 - val_loss: 1.6346 - val_accuracy: 0.5184
Epoch 2/30
8898/8898 [==============================] - 16s 2ms/step

 Phoneme Error Rate after epoch 1: 29.75906491679722%
2221/2221 [==============================] - 65s 29ms/step - loss: 1.5529 - accuracy: 0.5349 - val_loss: 1.5769 - val_accuracy: 0.5320
Epoch 3/30
8898/8898 [==============================] - 16s 2ms/step

 Phoneme Error Rate after epoch 2: 28.06010649325429%
2221/2221 [==============================] - 61s 27ms/step - loss: 1.4584 - accuracy: 0.5577 - val_loss: 1.4753 - val_accuracy: 0.5544
Epoch 4/30
8898/8898 [==============================] - 16s 2ms/step

 Phoneme Error Rate after epoch 3: 27.59337841845183%
2221/2221 [==============================] - 61s 27ms/step - loss: 1.4062 - accuracy: 0.5707 - val_

In [8]:
import json

# Collect training history metrics
training_metrics = {
    'epochs': list(range(1, len(history.history['accuracy']) + 1)),
    'accuracy': history.history['accuracy'],
    'loss': history.history['loss'],
    'val_acc': history.history['val_accuracy'],
    'val_loss': history.history['val_loss']
}

# Save the training metrics to a JSON file
with open('/kaggle/working/history_dnn.json', 'w') as file:
    json.dump(training_metrics, file, indent=4)
    
# Save the PER metrics to a JSON file
with open('/kaggle/working/per_dnn.json', 'w') as file:
    json.dump(performance_dnn, file, indent=4)

In [9]:
test_features_path = "/kaggle/input/timitpreprocessed/test_features.pkl"
test_labels_path = "/kaggle/input/timitpreprocessed/test_labels.pkl"

with open(test_features_path, 'rb') as pickle_file:
    test_features = pkl.load(pickle_file)
    
with open(test_labels_path, 'rb') as pickle_file:
    test_labels = pkl.load(pickle_file)

In [10]:
def compute_per(X, y, model, phoneme_mapping):
    # Get the model predictions
    predictions = model.predict(X)
    
    # Convert one-hot encoded vectors to phoneme sequences
    reference_phonemes = [one_hot_to_phoneme(vec, phoneme_mapping) for vec in y]
    predicted_phonemes = [one_hot_to_phoneme(vec, phoneme_mapping) for vec in predictions]

    # Calculate PER
    per = calculate_per(" ".join(reference_phonemes), " ".join(predicted_phonemes))
    
    return per

def one_hot_to_phoneme(one_hot_vector, phoneme_mapping):
    index = np.argmax(one_hot_vector)
    return phoneme_mapping.get(index-1, "")
    
def calculate_per(reference, hypothesis):
    distance = lev.distance(reference, hypothesis)
    per = distance / len(reference)
    return per * 100

In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(test_features, test_labels)

16236/16236 [==============================] - 45s 3ms/step - loss: 1.3999 - accuracy: 0.5828


In [12]:
# Compute PER on the test data
per = compute_per(test_features, test_labels,model, phoneme39_to_label)

print(f'Loss: {loss}, Accuracy: {accuracy}, Phoneme Error Rate: {per}%')

16236/16236 [==============================] - 29s 2ms/step
Loss: 1.399860143661499, Accuracy: 0.5827611684799194, Phoneme Error Rate: 26.453432675422484%


In [13]:
# Save the model as an HDF5 file
model.save("/kaggle/working/dnn_model.h5")